In [1]:
import mlrose_hiive
import numpy as np
import logging
import networkx as nx
import matplotlib.pyplot as plt
import string


from ast import literal_eval


from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from mlrose_hiive import QueensGenerator, MaxKColorGenerator, TSPGenerator
from mlrose_hiive import SARunner, GARunner, NNGSRunner
import pandas as pd

In [2]:
data = pd.read_csv('../Data/feature_engineered.csv').sample(100000, random_state = 2)
data = data.sample(500, random_state = 2)
predictors = ['Saturday', 'Tuesday_Wednesday', 'CARRIER_rank', 'ORIGIN_rank',
       'DEST_rank', 'LATE_NIGHT', 'DEP_HOUR_rank', 'DEP_DEL15']
target = 'ARR_DEL15'

In [3]:
target = np.array(data.loc[:,target])


In [4]:
target

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [5]:
# Load and Split data into training and test sets



X_train, X_test, y_train, y_test = train_test_split(
    data[predictors], target, test_size = 0.2, random_state = 2
)

# Normalize feature data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# One hot encode target values
one_hot = OneHotEncoder()
y_train_hot = one_hot.fit_transform(y_train.reshape(-1, 1)).todense()
y_test_hot = one_hot.transform(y_test.reshape(-1, 1)).todense()



## random hill climb parameter tuning 

In [4]:
grid_search_parameters = {
    'max_iters': [1000],                          # nn params
    'learning_rate': [1e-2],                       # nn params
    'activation': [mlrose_hiive.relu],            # nn params
    'restarts': [1,3, 10]                             # rhc params
}

nnr = NNGSRunner(
    x_train=X_train_scaled,
    y_train=y_train_hot,
    x_test=X_test_scaled,
    y_test=y_test_hot,
    experiment_name='nn_test_rhc',
    algorithm=mlrose_hiive.algorithms.rhc.random_hill_climb,
    grid_search_parameters=grid_search_parameters,
    iteration_list=[1, 10, 50, 100, 250, 500, 1000],
    hidden_layer_sizes=[[100]],
    bias=True,
    early_stopping=True,
    clip_max=5,
    max_attempts=500,
    n_jobs=5,
    seed=2,
    output_directory=None
)


run_stats_df, curves_df, cv_results_df, grid_search_cv = nnr.run()

NameError: name 'X_train_scaled' is not defined

In [12]:
grid_search_cv.best_params_ 

{'activation': <function mlrose_hiive.neural.activation.relu.relu(x, deriv=False)>,
 'hidden_layer_sizes': [100],
 'learning_rate': 0.01,
 'max_iters': 1000,
 'restarts': 1}

## SA parameter tuning 

In [10]:
grid_search_parameters = {
    'max_iters': [1000],                          # nn params
    'learning_rate': [1e-2],                       # nn params
    'activation': [mlrose_hiive.relu],            # nn params
    'schedule' : [mlrose_hiive.GeomDecay(init_temp=1, decay=0.5, min_temp=.001), mlrose_hiive.GeomDecay(init_temp=1, decay=0.3, min_temp=.001), mlrose_hiive.GeomDecay()]
}

nnr = NNGSRunner(
    x_train=X_train_scaled,
    y_train=y_train_hot,
    x_test=X_test_scaled,
    y_test=y_test_hot,
    experiment_name='nn_test_rhc',
    algorithm=mlrose_hiive.algorithms.sa.simulated_annealing,
    grid_search_parameters=grid_search_parameters,
    iteration_list=[1, 10, 50, 100, 250, 500, 1000],
    hidden_layer_sizes=[[100]],
    bias=True,
    early_stopping=True,
    clip_max=5,
    max_attempts=500,
    n_jobs=5,
    seed=2,
    output_directory=None
)


run_stats_df, curves_df, cv_results_df, grid_search_cv = nnr.run()

Fitting 5 folds for each of 3 candidates, totalling 15 fits


In [11]:
grid_search_cv.best_params_ 

{'activation': <function mlrose_hiive.neural.activation.relu.relu(x, deriv=False)>,
 'hidden_layer_sizes': [100],
 'learning_rate': 0.01,
 'max_iters': 1000,
 'schedule': GeomDecay(init_temp=1, decay=0.3, min_temp=0.001)}

## GA parameter tuning

In [7]:
grid_search_parameters = {
    'max_iters': [1000],                          # nn params
    'learning_rate': [0.001],                       # nn params
    'activation': [mlrose_hiive.relu],            # nn params
    'mutation_prob': [0.1, 0.25]# ga params
}

nnr = NNGSRunner(
    x_train=X_train_scaled,
    y_train=y_train_hot,
    x_test=X_test_scaled,
    y_test=y_test_hot,
    experiment_name='nn_test_rhc',
    algorithm=mlrose_hiive.algorithms.ga.genetic_alg,
    grid_search_parameters=grid_search_parameters,
    iteration_list=[1, 10, 50, 100, 250, 500, 1000],
    hidden_layer_sizes=[[100]],
    bias=True,
    early_stopping=True,
    clip_max=5,
    max_attempts=500,
    n_jobs=3,
    seed=2,
    output_directory=None
)


run_stats_df, curves_df, cv_results_df, grid_search_cv = nnr.run()

Fitting 5 folds for each of 2 candidates, totalling 10 fits


In [8]:
grid_search_cv.best_params_ 

{'activation': <function mlrose_hiive.neural.activation.relu.relu(x, deriv=False)>,
 'hidden_layer_sizes': [100],
 'learning_rate': 0.001,
 'max_iters': 1000,
 'mutation_prob': 0.25}